# 1 Introducción

Las funciones BLAS( Basic Linear Algebra Subprograms) es una especificación que define un conjunto de rutinas de bajo nivel para realizar operaciones comunes del álgebra lineal, como la suma de vectores, multiplicación escalar, combinaciones lineales, y multiplicación de matrices.

Una de estas funciones es la función swap. Esta funcion se encarga de intercambiar los valores entre dos vectores del mismo tamaño. Es decir los valores de un vector x pasarán al vector y, y los valores del vector y pasarán al vector x.

Para su implementación en cpu se hizo uso de un for donde se recorre el vector x, se guarda el elemento que se va a intercambiar en un auxiliar, y luego se pisa con el valor del elemento en y. Finalmente se guarda en y el valor del auxiliar.

Para su implementacion en gpu el algoritmo es similar, solo que en lugar de usar un for se utiliza hilos, donde podremos ver que tendremos una ventaja en el tiempo de ejecucion

Dicha funcion swap recibe 1 parametro en su implementacion en cpu. Este parametro es la cantidad de elementos en los vectores.

Para su implementacion en GPU la funcion que se va a ejecutar en el mismo recibe como parámetro un entero con la cantidad de elementos de los vectores, y dos punteros, uno al vector x y el otro al vector y.






# 2 Armado del ambiente
Instalar en el cuaderno el módulo CUDA de Python

In [1]:
!pip install pycuda

     |████████████████████████████████| 1.6MB 28.6MB/s 
     |████████████████████████████████| 71kB 12.3MB/s 
     |████████████████████████████████| 81kB 13.4MB/s 
  Created wheel for pycuda: filename=pycuda-2020.1-cp36-cp36m-linux_x86_64.whl size=621008 sha256=fa843c29b30e78e3042bf31de232f420c692c67096da2c48b745c39528c5df90
  Stored in directory: /root/.cache/pip/wheels/8f/78/d1/5bb826f81d9d490297a348d818ff3ee6dd6f2075b06dde6ea0
  Created wheel for pytools: filename=pytools-2020.4.3-py2.py3-none-any.whl size=61374 sha256=3dc86adafa1c7d09ba850b9acc1bc92d048da8a3a68739ff5c0ce3f83f3e666c
  Stored in directory: /root/.cache/pip/wheels/af/c7/81/a22edb90b0b09a880468b2253bb1df8e9f503337ee15432c64
Successfully built pycuda pytools


#3 Desarrollo

En CPU

In [2]:
# --------------------------------------------
#@title 3.1 Parámetros de ejecución { vertical-output: true }

n_elementos =   5000#@param {type: "number"}

# --------------------------------------------

from datetime import datetime

tiempo_total = datetime.now()

import numpy

# --------------------------------------------
# Definición de función que transforma el tiempo en  milisegundos 
tiempo_en_ms = lambda dt:(dt.days * 24 * 60 * 60 + dt.seconds) * 1000 + dt.microseconds / 1000.0

# --------------------------------------------
# CPU - Defino la memoria de los vectores en cpu.
vec_x_cpu = numpy.random.randn( n_elementos )
vec_x_cpu = vec_x_cpu.astype( numpy.float32() )

vec_y_cpu = numpy.random.randn( n_elementos )
vec_y_cpu = vec_y_cpu.astype( numpy.float32() )

#muestro los vectores antes de intercambiarlo

print("------------------------------------")
print( "Vectores antes del intercambio" )
print( "Vector X" )
print(vec_x_cpu)
print( "Vector Y" )
print(vec_y_cpu)
print("------------------------------------")

# CPU - Realizo la función swap.
tiempo_bucle = datetime.now()

for i in range( 0, n_elementos ):
  aux = vec_x_cpu[i]
  vec_x_cpu[i] = vec_y_cpu[i]
  vec_y_cpu[i] = aux

tiempo_bucle = datetime.now() - tiempo_bucle

# --------------------------------------------

print("------------------------------------")
print( "Vectores despues del intercambio" )
print( "Vector X" )
print(vec_x_cpu)
print( "Vector Y" )
print(vec_y_cpu)
print("------------------------------------")


tiempo_total = datetime.now() - tiempo_total

print("Tiempo Total: ", tiempo_en_ms( tiempo_total ), "[ms]" )
print("Tiempo bucle: ", tiempo_en_ms( tiempo_bucle ), "[ms]" )

------------------------------------
Vectores antes del intercambio
Vector X
[ 0.5714745  -0.7977685   0.04660711 ...  2.4777863   0.50267667
  0.08156705]
Vector Y
[ 1.3539588   0.2386185  -0.80707806 ... -0.69747895 -1.7778789
 -0.792623  ]
------------------------------------
------------------------------------
Vectores despues del intercambio
Vector X
[ 1.3539588   0.2386185  -0.80707806 ... -0.69747895 -1.7778789
 -0.792623  ]
Vector Y
[ 0.5714745  -0.7977685   0.04660711 ...  2.4777863   0.50267667
  0.08156705]
------------------------------------
Tiempo Total:  16.239 [ms]
Tiempo bucle:  9.664 [ms]


En GPU

In [3]:
# --------------------------------------------
#@title 3.1 Parámetros de ejecución { vertical-output: true }

n_elementos =   5000#@param {type: "number"}
# --------------------------------------------

from datetime import datetime

tiempo_total = datetime.now()

import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

import numpy

# --------------------------------------------
# Definición de función que transforma el tiempo en  milisegundos 
tiempo_en_ms = lambda dt:(dt.days * 24 * 60 * 60 + dt.seconds) * 1000 + dt.microseconds / 1000.0


# CPU - Defino la memoria de los vectores en cpu.
vec_x_cpu = numpy.random.randn( n_elementos )
vec_x_cpu = vec_x_cpu.astype( numpy.float32() )

vec_y_cpu = numpy.random.randn( n_elementos )
vec_y_cpu = vec_y_cpu.astype( numpy.float32() )

#muestro los vectores antes de intercambiarlo
print("------------------------------------")
print( "Vectores antes del intercambio" )
print( "Vector X" )
print(vec_x_cpu)
print( "Vector Y" )
print(vec_y_cpu)
print("------------------------------------")


# CPU - reservo la memoria GPU.
vec_x_gpu = cuda.mem_alloc( vec_x_cpu.nbytes )
vec_y_gpu = cuda.mem_alloc( vec_y_cpu.nbytes )

# GPU - Copio la memoria al GPU.
cuda.memcpy_htod( vec_x_gpu, vec_x_cpu )
cuda.memcpy_htod( vec_y_gpu, vec_y_cpu )

# CPU - Defino la función kernel que ejecutará en GPU.
module = SourceModule("""
__global__ void kernel_swap( int n, float *X, float *Y )
{
  int idx = threadIdx.x + blockIdx.x*blockDim.x;
  float aux;
  if( idx < n )
  {
    aux = X[idx];
    X[idx] = Y[idx];
    Y[idx] = aux;
  }
}
""") 
# CPU - Genero la función kernel.
kernel = module.get_function("kernel_swap")

tiempo_gpu = datetime.now()

# GPU - Ejecuta el kernel.
dim_hilo = 256
dim_bloque = numpy.int( (n_elementos+dim_hilo-1) / dim_hilo )
print( "Thread x: ", dim_hilo, ", Bloque x:", dim_bloque )

kernel( numpy.int32(n_elementos), vec_x_gpu, vec_y_gpu, block=( dim_hilo, 1, 1 ),grid=(dim_bloque, 1,1) )

tiempo_gpu = datetime.now() - tiempo_gpu

# GPU - Copio el resultado desde la memoria GPU.
cuda.memcpy_dtoh( vec_x_cpu, vec_x_gpu )
cuda.memcpy_dtoh( vec_y_cpu, vec_y_gpu )

print("------------------------------------")
print( "Vectores despues del intercambio" )
print( "Vector X" )
print(vec_x_cpu)
print( "Vector Y" )
print(vec_y_cpu)
print("------------------------------------")

tiempo_total = datetime.now() - tiempo_total

print( "Cantidad de elementos: ", n_elementos )
print( "Thread x: ", dim_hilo, ", Bloque x:", dim_bloque )
print("Tiempo CPU: ", tiempo_en_ms( tiempo_total ), "[ms]" )
print("Tiempo GPU: ", tiempo_en_ms( tiempo_gpu   ), "[ms]" )

------------------------------------
Vectores antes del intercambio
Vector X
[ 0.14898564  0.99304354 -1.3121908  ...  0.11165223  1.8231604
 -0.7710918 ]
Vector Y
[ 0.32859913  2.7047126   0.08579163 ... -0.98051226  0.16326524
 -0.2667367 ]
------------------------------------
Thread x:  256 , Bloque x: 20
------------------------------------
Vectores despues del intercambio
Vector X
[ 0.32859913  2.7047126   0.08579163 ... -0.98051226  0.16326524
 -0.2667367 ]
Vector Y
[ 0.14898564  0.99304354 -1.3121908  ...  0.11165223  1.8231604
 -0.7710918 ]
------------------------------------
Cantidad de elementos:  5000
Thread x:  256 , Bloque x: 20
Tiempo CPU:  1136.436 [ms]
Tiempo GPU:  2.457 [ms]


# 4 Tabla de pasos

# 4.1 En CPU

Procesador | Función | Detalle
---------- | -------   | -------
CPU        | @param   | Tamaño del los vecotores x e y
CPU        | import | Importa los modulos necesarios para el correcto funcionamiento
CPU        | datetime.now() |	Toma el tiempo inicial.
CPU        | numpy.random.randn( n_elementos ) | Inicializa los vectores x e y
CPU        | for...| Realiza el intercambio entre los vectores x e y
CPU        | datetime.now() | Toma el tiempo final
CPU        | print() | Muestra los vectores intercambiados por pantalla
CPU        | print() | Muestra los resultados de la ejecución por pantalla



# 4.2 En GPU
Procesador | Función | Detalle
---------- | -------   | -------
CPU        | @param   | 	Lectura del tamaño de vectores x e y desde Colab
CPU        | import | Importa los modulos necesarios para el correcto funcionamiento
CPU        | datetime.now() |	Toma el tiempo inicial.
CPU        | numpy.random.randn( n_elementos ) | Inicializa los vectores x e y
GPU        | cuda.mem_alloc() | Reserva la memoria en GPU
GPU        | cuda.memcpy_htod() | Copia las memorias desde el CPU al GPU.
CPU        | SourceModule() |	Define el código del kernel
CPU        | module.get_function() | Genera la función del kernel GPU
CPU        | dim_tx/dim_bx | Calcula las dimensiones
GPU        | kernel() | Ejecuta el kernel en GPU
CPU        | cuda.memcpy_dtoh( ) | Copia el resultado desde GPU memoria A a CPU memoria R.
CPU        | print() | Muestra los vectores intercambiados por pantalla
CPU        | print() | Muestra los resultados de la ejecución por pantalla

# 5 Conclusiones

Podemos ver que los tiempos de ejecucion utilizando CUDA, y un tamaño de vecotres de 5000 elementos, son mucho menor que utilizando la cpu de manera secuencial. Esto debido a que usamos la ventaja que nos da la implementacion de hilos, ya que se generará uno por cada elemento del vector, acelerando el proceso de intercambio entre vectores. No siempre es mejor la implementacion en hilos, puede ser que haya operaciones que dependan de otras para terminar, y puede llegar a ocurrir el caso en el que una implementacion secuencial saca mejor provecho que en paralelo. Esto pasa en el caso donde la entrada ( la cantidad de elementos) no es tan grande.

Para CPU: 
Tiempo Total:  9.875 [ms]

Para GPU:
Tiempo Total:  6.420 [ms]

# 6 Bibliografía

[1] LBLAS Technical Forum: [Referencia](http://netlib.org/blas/blast-forum/)

[2] Developer Reference for Intel® Math Kernel Library - C: [Referencia](https://software.intel.com/content/www/us/en/develop/documentation/mkl-developer-reference-c/top/blas-and-sparse-blas-routines/blas-routines/blas-level-1-routines-and-functions.html)

[3] GPU Accelerated Computing with Python: [Referencia](https://developer.nvidia.com/how-to-cuda-python)

[4] PyCUDA Documentation: [Referencia](https://documen.tician.de/pycuda/)